In [ ]:
import numpy as np
import pandas as pd
!pip install tensorflow numpy pandas matplotlib


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [ ]:
image_size = (224, 224)

In [ ]:
batch_size = 32
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/Users/vimlendusharma/Downloads/Dataset for Crop Pest and Disease Detection/CCMT Dataset-Augmented/Cashew',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
validation_generator = test_datagen.flow_from_directory(
    '/Users/vimlendusharma/Downloads/Dataset for Crop Pest and Disease Detection/CCMT Dataset-Augmented/Cashew',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

In [ ]:
from PIL import Image
import os

def check_images(image_dir):
    for root, _, files in os.walk(image_dir):
        for file in files:
            try:
                img_path = os.path.join(root, file)
                with Image.open(img_path) as img:
                    img.verify()  # Verify if it's a valid image
            except (IOError, SyntaxError) as e:
                print(f"Error with file {img_path}: {e}")

check_images('/Users/vimlendusharma/Downloads/Dataset for Crop Pest and Disease Detection')


In [ ]:
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

model.save('crop_disease_detection_model.h5')